In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import plaidml.keras
plaidml.keras.install_backend()

In [2]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [3]:
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./MINST_data/", one_hot=True)
x_train = mnist.train.images
x_test = mnist.test.images
y_train = mnist.train.labels.astype(np.float32)
y_test = mnist.test.labels.astype(np.float32)

W0724 11:40:25.101354  6116 deprecation.py:323] From <ipython-input-3-23ce2e5d135f>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0724 11:40:25.109089  6116 deprecation.py:323] From C:\Coding\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0724 11:40:25.110538  6116 deprecation.py:323] From C:\Coding\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions 

Extracting ./MINST_data/train-images-idx3-ubyte.gz


W0724 11:40:25.670172  6116 deprecation.py:323] From C:\Coding\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0724 11:40:25.677844  6116 deprecation.py:323] From C:\Coding\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0724 11:40:25.796456  6116 deprecation.py:323] From C:\Coding\Anaconda\envs\tensorflow\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and wi

Extracting ./MINST_data/train-labels-idx1-ubyte.gz
Extracting ./MINST_data/t10k-images-idx3-ubyte.gz
Extracting ./MINST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print('x_train shape:', x_train.shape)
print('y_train shape', y_train.shape)

x_train shape: (55000, 28, 28, 1)
y_train shape (55000, 10)


In [5]:
batch_size = 512
num_classes = 10
epochs = 15

In [6]:
# convert class vectors to binary class matrices; not needed here as the data is already in one-hot
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [7]:
# TensorFlow uses the probability to keep nodes, while Keras uses the probability to drop them.
def simple_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def CNN_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
    return model

In [8]:
model = simple_model()

INFO:plaidml:Opening device "opencl_nvidia_geforce_gtx_1050.0"
I0724 11:40:56.782687  6116 library.py:135] Opening device "opencl_nvidia_geforce_gtx_1050.0"


In [9]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs, verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)

Train on 55000 samples, validate on 10000 samples
Epoch 1/15
54784/55000 [============================>.] - ETA: 0s - loss: 0.5486 - acc: 0.8279

INFO:plaidml:Analyzing Ops: 83 of 282 operations complete
I0724 11:41:13.044248  6116 library.py:135] Analyzing Ops: 83 of 282 operations complete


55000/55000 [==============================] - 16s 282us/step - loss: 0.5470 - acc: 0.8284 - val_loss: 0.1151 - val_acc: 0.9659
Epoch 2/15
55000/55000 [==============================] - 7s 119us/step - loss: 0.1561 - acc: 0.9537 - val_loss: 0.0857 - val_acc: 0.9722
Epoch 3/15
55000/55000 [==============================] - 6s 107us/step - loss: 0.1112 - acc: 0.9663 - val_loss: 0.0555 - val_acc: 0.9817
Epoch 4/15
55000/55000 [==============================] - 5s 89us/step - loss: 0.0918 - acc: 0.9723 - val_loss: 0.0453 - val_acc: 0.9841
Epoch 5/15
55000/55000 [==============================] - 5s 88us/step - loss: 0.0770 - acc: 0.9770 - val_loss: 0.0444 - val_acc: 0.9858
Epoch 6/15
55000/55000 [==============================] - 5s 88us/step - loss: 0.0665 - acc: 0.9805 - val_loss: 0.0375 - val_acc: 0.9881
Epoch 7/15
55000/55000 [==============================] - 5s 87us/step - loss: 0.0612 - acc: 0.9809 - val_loss: 0.0382 - val_acc: 0.9874
Epoch 8/15
55000/55000 [========================

In [10]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.030447034907341002
Test accuracy: 0.9898
